# Batch insert into database

In [1]:
import psycopg2
from twelvedata import TDClient
from psycopg2.extras import execute_values
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

In [2]:
# Load environment variables from .env file
load_dotenv()

# Get environment variables
tdapikey = os.getenv('TDAPIKEY')
dbname, host, user, password, port = os.getenv('DBNAME'), os.getenv('HOST'), os.getenv('DBUSER'), os.getenv('PASS'), os.getenv('PORT')

# Connect to the database
conn = psycopg2.connect(database=dbname, host=host, user=user, password=password, port=port)
cursor = conn.cursor()

In [9]:
# Define the table name and columns
symbol = "AAPL"
table_name = "stocks_real_time"
columns = ["time", "symbol", "price", "day_volume"]

# Define the start and end dates for the data
end_date = datetime.now().strftime("%Y-%m-%d")
start_date = (datetime.now() - timedelta(days=2 * 365)).strftime("%Y-%m-%d")

In [10]:
# Fetch data from Twelve Data API
td = TDClient(apikey=tdapikey)
time_series = td.time_series(symbol=symbol, interval="1h", start_date=start_date, end_date=end_date, outputsize=5000)
data = time_series.as_pandas()

In [11]:
data

,open,high,low,close,volume
datetime,,,,,
2023-07-14 15:30:00,190.27499,190.74001,190.1600,190.73000,5002552
2023-07-14 14:30:00,190.20000,190.42000,189.7800,190.27499,4052940
2023-07-14 13:30:00,190.01010,190.56500,189.9501,190.19000,3213789
2023-07-14 12:30:00,189.95000,190.14999,189.6300,190.01500,3352688
2023-07-14 11:30:00,190.36501,190.62000,189.8900,189.95000,3952148
...,...,...,...,...,...
2021-07-19 13:30:00,141.78000,142.55000,141.7000,142.53000,9473277
2021-07-19 12:30:00,142.91000,143.02000,141.7300,141.78999,11005192
2021-07-19 11:30:00,143.20000,143.95000,142.8000,142.90500,12189392


In [12]:
# Convert the data to a list of tuples for batch insert
#data_tuples = [(row.Index.to_pydatetime(), row['symbol'], row['close'], row['volume']) for row in data.itertuples()]
data_tuples = [(row.Index.to_pydatetime(), symbol, row.close, row.volume) for row in data.itertuples()]

In [14]:
# Perform the batch insert into the database
execute_values(cursor, f"INSERT INTO {table_name} ({','.join(columns)}) VALUES %s", data_tuples)
conn.commit()

In [15]:
# Close the database connection
cursor.close()
conn.close()